In [10]:
import os
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key

'73bdec39-1f93-47fc-bd2f-f02883d7be83'

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Data vectorization



## Load data


In [12]:
pdf_path = 'data_source'
loader = DirectoryLoader(path=pdf_path, glob='*.pdf', loader_cls=PyPDFLoader)
documents = loader.load()

## Data split

In [13]:
separators = [
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma
    "\u3001",  # Ideographic comma
    "\uff0e",  # Fullwidth full stop
    "\u3002",  # Ideographic full stop
    "",
]
chunk_size = 500
chunk_overlap = 50

def word_count(text):
    return len(text.split())

In [14]:
#splitter
document_splitter = RecursiveCharacterTextSplitter(
    separators = separators,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
    is_separator_regex = False
)
#embedding
#vector_store

In [15]:
chunks = document_splitter.split_documents(documents)
chunks

[Document(metadata={'source': 'data_source\\5_nguyen_nhan_gay_tram_cam_sau_sinh__nhan_biet_som_de_dieu_tri_kip_thoi.pdf', 'page': 0}, page_content='5 nguyên nhân gây trầm cảm sau sinh - nhận biết sớm để điều trị kịp thời\nBài viết được tư vấn chuyên môn bởi Bác sĩ chuyên khoa II Trần Thị Mai Hương - Khoa sản\nphụ khoa - Bệnh viện Đa khoa Quốc tế Vinmec Hải Phòng\nTâm lý phụ nữ sau sinh bị ảnh hưởng, kết hợp nhiều yếu tố tác động gây nên chứng trầm cảm\nsau sinh. Nếu không được phát hiện và có phương pháp điều trị tâm lý đúng cách thì chứng\ntrầm cảm sau sinh có thể gây ra những hậu quả đáng tiếc và nguy hiểm đến tính mạng của cả\nmẹ và con.'),
 Document(metadata={'source': 'data_source\\5_nguyen_nhan_gay_tram_cam_sau_sinh__nhan_biet_som_de_dieu_tri_kip_thoi.pdf', 'page': 0}, page_content='mẹ và con.\n1. Trầm cảm sau sinh là gì?\nTrầm cảm sau sinh là tình trạng liên quan đến suy nghĩ và cảm giác mệt mỏi, buồn chán, lo\nlắng,  tuyệt  vọng  xuất  hiện  sau  sinh.  Trầm  cảm  sau  sinh  có

## Embedding model

In [16]:
embedding_model = HuggingFaceEmbeddings()

c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
embedded_txt = embedding_model.embed_query('Các triệu chứng rối loạn tâm thần chu sinh có thể bao gồm những gì?')
embedded_txt

[0.031750041991472244,
 -0.06335990875959396,
 0.010337479412555695,
 -0.009711507707834244,
 0.008868977427482605,
 0.05885063111782074,
 -0.0434853620827198,
 0.02191467583179474,
 0.03367353230714798,
 0.012118174694478512,
 -0.06679972261190414,
 -0.01079217717051506,
 0.07173742353916168,
 -0.002301837084814906,
 0.005899115465581417,
 0.013180364854633808,
 0.03142682462930679,
 -0.009340452030301094,
 0.017850961536169052,
 -0.05089427903294563,
 -0.022570393979549408,
 0.0042210835963487625,
 0.010375314392149448,
 0.06504923850297928,
 0.004980253521353006,
 -0.059181638062000275,
 0.011899792589247227,
 -0.035085298120975494,
 -0.0009733123588375747,
 -0.014011919498443604,
 0.060917459428310394,
 0.007390104234218597,
 0.0030016887467354536,
 0.0023666806519031525,
 2.1575706341536716e-06,
 -0.03721456602215767,
 -0.023524394258856773,
 -0.020614242181181908,
 -0.07700739055871964,
 0.04336024448275566,
 0.045382287353277206,
 0.07151366770267487,
 0.0039085266180336475,
 0.

## Vector store


In [17]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index('docs-rag-chatbot')

In [18]:
db = PineconeVectorStore.from_documents(documents=chunks, embedding=embedding_model, index_name="docs-rag-chatbot", namespace="docs-store" )

## Query vector database



### Create vectordb class

In [1]:
class VectorDB:
    def __init__(self):
        self.embedding = HuggingFaceEmbeddings()
        self.db = self.__build_db__()

    def __build_db__(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-store",
            embedding=self.embedding
        )
        return db
    
    def get_retriever(self,
                      search_type: str = 'similarity', 
                      search_kwargs: dict = {'k': 10}):
        retriever = self.db.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
        return retriever


